In [71]:
from gurobipy import Model, GRB

In [72]:
# Create a new model
m = Model("bike_redistribution")

In [73]:
# Define the sets for hubs and edges including costs
edges_with_costs = {
    (1, 2): 20, (1, 5): 25,
    (2, 5): 30, (2, 7): 45,
    (3, 1): 20, (3, 6): 35,
    (4, 2): 30,
    (5, 3): 25, (5, 4): 15, (5, 6): 28,
    (6, 7): 12,
    (7, 4): 27
}

In [74]:
# Create variables for each edge
x = {}
for i, j in edges_with_costs.keys():
    x[i, j] = m.addVar(vtype=GRB.INTEGER, name=f"x_{i}_{j}")

In [75]:
# Add hub imbalance constraints
m.addConstr(x[1, 2] + x[1, 5] - x[3, 1] == -112, "Hub1_Balance")
m.addConstr(x[2, 5] + x[2, 7] - x[1, 2] - x[4, 2] == 70, "Hub2_Balance")
m.addConstr(x[3, 1] + x[3, 6] - x[5, 3] == 42, "Hub3_Balance")
m.addConstr(x[4, 2] - x[5, 4] - x[7, 4] == -42, "Hub4_Balance")
m.addConstr(x[5, 3] + x[5, 4] + x[5, 6] - x[1, 5] - x[2, 5] == 28, "Hub5_Balance")
m.addConstr(x[6, 7] - x[3, 6] - x[5, 6] == -70, "Hub6_Balance")
m.addConstr(x[7, 4] - x[2, 7] - x[6, 7] == 84, "Hub7_Balance")

<gurobi.Constr *Awaiting Model Update*>

In [76]:
# Add transfer limit constraint
m.addConstr(x[2, 5] + x[2, 7] + x[3, 1] + x[3, 6] <= 2 * (x[4, 2] + x[5, 3] + x[5, 4] + x[5, 6]), "Transfer_Limit")

<gurobi.Constr *Awaiting Model Update*>

In [77]:
# Constraints for transfers between hubs 1-5
m.addConstr(
    sum(x[i, j] for i in range(1, 6) for j in range(1, 6) if (i, j) in edges_with_costs) >= 0.05 * 1400
)
m.addConstr(
    sum(x[i, j] for i in range(1, 6) for j in range(1, 6) if (i, j) in edges_with_costs) <= 0.50 * 1400
)

<gurobi.Constr *Awaiting Model Update*>

In [78]:
# Add non-negativity constraints for all decision variables
for i, j in edges_with_costs.keys():
    m.addConstr(x[i, j] >= 0, f"Non-negativity_{i}_{j}")

In [79]:
# Explicit Objective Function
m.setObjective(sum(edges_with_costs[i, j] * x[i, j] for i, j in edges_with_costs), GRB.MINIMIZE)

In [80]:
# Optimize the model
m.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 22 rows, 12 columns and 58 nonzeros
Model fingerprint: 0x5d08e1fb
Variable types: 0 continuous, 12 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+01, 4e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [3e+01, 7e+02]
Found heuristic solution: objective 15988.000000
Presolve removed 15 rows and 2 columns
Presolve time: 0.00s
Presolved: 7 rows, 10 columns, 32 nonzeros
Variable types: 0 continuous, 10 integer (0 binary)

Root relaxation: objective 1.283800e+04, 2 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0    12838.000000 12838.0000  0.00%     -    0s

In [81]:
# Output the solution
if m.status == GRB.Status.OPTIMAL:
    solution = m.getAttr('x', x)
    for i, j in edges_with_costs.keys():
        if solution[i, j] > 0:
            print(f"Transfer {solution[i, j]} bikes from hub {i} to hub {j}")

Transfer 112.0 bikes from hub 2 to hub 5
Transfer 112.0 bikes from hub 3 to hub 1
Transfer 42.0 bikes from hub 4 to hub 2
Transfer 70.0 bikes from hub 5 to hub 3
Transfer 70.0 bikes from hub 5 to hub 6
Transfer 84.0 bikes from hub 7 to hub 4


In [82]:
# Output the solution
if m.status == GRB.Status.OPTIMAL:
    solution = m.getAttr('x', x)
    for (i, j) in edges_with_costs:
        if solution[i, j] > 0:
            print(f"Transfer {solution[i, j]} bikes from hub {i} to hub {j}")
else:
    print('No optimal solution found')

Transfer 112.0 bikes from hub 2 to hub 5
Transfer 112.0 bikes from hub 3 to hub 1
Transfer 42.0 bikes from hub 4 to hub 2
Transfer 70.0 bikes from hub 5 to hub 3
Transfer 70.0 bikes from hub 5 to hub 6
Transfer 84.0 bikes from hub 7 to hub 4
